Монитрую диск.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import datetime

Папка с проектом находится по адресу:

In [ ]:
project_dir = '/content/drive/My Drive/Colab Notebooks/lkl2022/Q10/'

Устанавливаю программу для формирования отчёта анализа данных.  
pandas-profiling

In [ ]:
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

In [ ]:
import pandas_profiling

In [ ]:
from pandas_profiling import ProfileReport

Загружаю изначальный датасет. Я только переименовал русское название файла, на английское, данные, как в ТЗ.

In [ ]:
data_  = project_dir  + 'data/roh_v10.xlsx'
data_roh_ = pd.read_excel(data_)

Вывожу пять значений.

In [ ]:
data_roh_.sample(5)

,№ - ТОП,№,Адрес,Площадь м2,Год,Этажей,Подьездов,Помещений / Квартир
11881,NaN,25191.0,"ул. Русаковская, д. 25, Москва",12003.00,1971,12,NaN,NaN
11570,NaN,22631.0,"ул. Первомайская, д. 51/23, Москва",4567.50,1970,12,NaN,NaN
27161,NaN,22988.0,"ул. Петровка, д. 26, стр. 5, Москва",2783.00,1877,5,NaN,NaN
30499,NaN,29403.0,"ул. Флотская, д. 82/6, Москва",2972.60,1957,4,NaN,NaN
9036,NaN,10685.0,"ул. Дегунинская, д. 12, к. 1, Москва",4340.80,2015,13,NaN,NaN


In [ ]:
data_roh_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33398 entries, 0 to 33397
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   № - ТОП              1923 non-null   float64
 1   №                    32157 non-null  float64
 2   Адрес                33398 non-null  object 
 3   Площадь м2           33398 non-null  object 
 4   Год                  33398 non-null  object 
 5   Этажей               33398 non-null  object 
 6   Подьездов            4072 non-null   object 
 7   Помещений / Квартир  4062 non-null   object 
dtypes: float64(2), object(6)
memory usage: 2.0+ MB


In [ ]:
data_roh_['№ - ТОП'].value_counts()

1.0       1
518.0     1
1803.0    1
1801.0    1
1800.0    1
         ..
722.0     1
721.0     1
720.0     1
719.0     1
1917.0    1
Name: № - ТОП, Length: 1923, dtype: int64

In [ ]:
data_roh_.columns

Index(['№ - ТОП', '№', 'Адрес', 'Площадь м2', 'Год', 'Этажей', 'Подьездов',
       'Помещений / Квартир'],
      dtype='object')

Начнём менять данные.  
Сделаем копию датасета.  
Увидим, что кроме NaN-значений существуют не явно указанные значения. Их надо перевести в NaN-значения. Не обязательно запоминать. В некоторых ячейках записали либо 'не указано', 'не заполнено' и т.п. Метод value_counts() поможет найти (или вывести все строковые ячейки где должны быть числовые...) и поменяем на NaN значения. Только после этого стоит делать pandas profilling с первоначальным датасетом.

In [ ]:
data_roh = data_roh_.copy()

In [ ]:
new_columns = ['number_top', 'number', 'address', 'area_sqm', 'year', 'floors', 'entrances',
       'apartments_flats']

In [ ]:
data_roh.columns = new_columns

In [ ]:
data_roh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33398 entries, 0 to 33397
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   number_top        1923 non-null   float64
 1   number            32157 non-null  float64
 2   address           33398 non-null  object 
 3   area_sqm          33398 non-null  object 
 4   year              33398 non-null  object 
 5   floors            33398 non-null  object 
 6   entrances         4072 non-null   object 
 7   apartments_flats  4062 non-null   object 
dtypes: float64(2), object(6)
memory usage: 2.0+ MB


**number_top**

In [ ]:
data_roh['number_top'].value_counts()

1.0       1
518.0     1
1803.0    1
1801.0    1
1800.0    1
         ..
722.0     1
721.0     1
720.0     1
719.0     1
1917.0    1
Name: number_top, Length: 1923, dtype: int64

In [ ]:
data_roh[data_roh['number_top'].isnull()]

,number_top,number,address,area_sqm,year,floors,entrances,apartments_flats
1389,NaN,260.0,"г. Зеленоград, к. 2005, Москва",29695.20,2007,17,NaN,NaN
1390,NaN,262.0,"г. Зеленоград, к. 2010, Москва",29628.60,2006,17,8,400
1391,NaN,265.0,"г. Зеленоград, к. 2016, Москва",24855.00,2006,17,NaN,NaN
1392,NaN,269.0,"г. Зеленоград, к. 2024, Москва",29834.60,2006,17,NaN,NaN
1393,NaN,271.0,"г. Зеленоград, к. 2028, Москва",30031.90,2007,17,7,400
...,...,...,...,...,...,...,...,...
33393,NaN,NaN,"ул. Оренбургская, д. 5, к. 4, Москва",107.10,1954,1,Не указано,2
33394,NaN,NaN,"г. Зеленоград, проезд. 6537-й, д. 21, Москва",469.00,2009,1,Не указано,Не указано
33395,NaN,NaN,"ул. Спортивная, д. 10, Москва",133.00,1940,1,Не указано,Не указано
33396,NaN,NaN,"проезд. Коломенский, д. 8, к. 1, Москва",8462.30,1982,0,2,128


**number**

In [ ]:
data_roh['number'].value_counts()

18803.0    1
2869.0     1
2926.0     1
2908.0     1
2904.0     1
          ..
14269.0    1
14265.0    1
14264.0    1
14263.0    1
30476.0    1
Name: number, Length: 32157, dtype: int64

In [ ]:
data_roh[data_roh['number'].isnull()]

,number_top,number,address,area_sqm,year,floors,entrances,apartments_flats
32157,NaN,NaN,"ул. Красногвардейская 3-я, д. 8, стр. 1, Москва",6160.80,1970,9,4,144
32158,NaN,NaN,"ул. Фрунзенская 3-я, д. 26, Москва",7972.00,1966,9,5,180
32159,NaN,NaN,"пер. Вятский 4-й, д. 33/47, Москва",2183.10,1960,9,1,64
32160,NaN,NaN,"проезд. Рощинский 5-й, д. 7/8, Москва",7168.20,1979,9,4,129
32161,NaN,NaN,"ул. Парковая 5-я, д. 26, Москва",5174.00,1965,9,3,96
...,...,...,...,...,...,...,...,...
33393,NaN,NaN,"ул. Оренбургская, д. 5, к. 4, Москва",107.10,1954,1,Не указано,2
33394,NaN,NaN,"г. Зеленоград, проезд. 6537-й, д. 21, Москва",469.00,2009,1,Не указано,Не указано
33395,NaN,NaN,"ул. Спортивная, д. 10, Москва",133.00,1940,1,Не указано,Не указано
33396,NaN,NaN,"проезд. Коломенский, д. 8, к. 1, Москва",8462.30,1982,0,2,128


**address**

In [ ]:
data_roh['address'].value_counts()

ул. Николая Старостина, д. 15, Москва       2
пр-кт. Федеративный, д. 5, к. 1, Москва     2
ул. Воротынская, д. 3, Москва               2
пер. Кадашевский 3-й, д. 4, Москва          2
проезд. Окружной, д. 13, Москва             2
                                           ..
ш. Каширское, д. 128, Москва                1
ш. Каширское, д. 126, Москва                1
ш. Каширское, д. 124, к. 1, Москва          1
ш. Каширское, д. 116, к. 1, Москва          1
б-р. Чистопрудный, д. 23, стр. 1, Москва    1
Name: address, Length: 32153, dtype: int64

In [ ]:
data_roh[data_roh['address'].isnull()] #нет нулевых значений

,number_top,number,address,area_sqm,year,floors,entrances,apartments_flats


**area_sqm**

In [ ]:
data_roh['area_sqm'].value_counts()

Не заполнено    183
0.00             24
3621.00          16
3635.00          16
3637.00          15
               ... 
4717.00           1
17449.40          1
17445.40          1
8867.70           1
4466.40           1
Name: area_sqm, Length: 24107, dtype: int64

In [ ]:
data_roh['area_sqm'].value_counts()[0]

1

In [ ]:
data_roh[data_roh['area_sqm'].isnull()] #нет нулевых значений

,number_top,number,address,area_sqm,year,floors,entrances,apartments_flats


In [ ]:
data_roh[data_roh['area_sqm'] == 'Не заполнено']

,number_top,number,address,area_sqm,year,floors,entrances,apartments_flats
25,27.0,13229.0,"ул. Исаковского, д. 39, к. 1, Москва",Не заполнено,2004,37,1,172
70,73.0,12750.0,"ул. Ивана Бабушкина, д. 10, Москва",Не заполнено,2008,31,1,143
105,110.0,25115.0,"ш. Рублевское, д. 101, Москва",Не заполнено,2019,28,1,879
658,747.0,28227.0,"ул. Твардовского, д. 14, к. 1, Москва",Не заполнено,2006,23,NaN,NaN
713,904.0,6565.0,"ул. Набережная Б., д. 9, Москва",Не заполнено,2004,22,2,Не указано
...,...,...,...,...,...,...,...,...
10877,NaN,15384.0,"б-р. Кронштадтский, д. 30, к. 3, Москва",Не заполнено,Не заполнено,12,NaN,NaN
11407,NaN,20875.0,"ул. Новолесная, д. 18, к. 2, Москва",Не заполнено,1966,12,NaN,NaN
12152,NaN,27341.0,"пер. Старопименовский, д. 12/6, Москва",Не заполнено,1973,12,NaN,NaN
15213,NaN,14108.0,"ул. Климашкина, д. 9, Москва",Не заполнено,Не заполнено,9,NaN,NaN


In [ ]:
data_roh.loc[25]['area_sqm']

'Не заполнено'

Внимание! Хоть нет NaN значений, надо их создать поскольку 183 ячейки содержат фразу 'Не заполнено'

In [ ]:
data_roh['area_sqm'].replace('Не заполнено', np.nan, inplace = True)

In [ ]:
data_roh['area_sqm'].value_counts()

0.00        24
3635.00     16
3621.00     16
3637.00     15
3639.00     15
            ..
4717.00      1
17449.40     1
17445.40     1
8867.70      1
4466.40      1
Name: area_sqm, Length: 24106, dtype: int64

Итак мы по месту заменили данные на NaN, все 183 значения.

In [ ]:
data_roh[data_roh['area_sqm'].isnull()]

,number_top,number,address,area_sqm,year,floors,entrances,apartments_flats
25,27.0,13229.0,"ул. Исаковского, д. 39, к. 1, Москва",NaN,2004,37,1,172
70,73.0,12750.0,"ул. Ивана Бабушкина, д. 10, Москва",NaN,2008,31,1,143
105,110.0,25115.0,"ш. Рублевское, д. 101, Москва",NaN,2019,28,1,879
658,747.0,28227.0,"ул. Твардовского, д. 14, к. 1, Москва",NaN,2006,23,NaN,NaN
713,904.0,6565.0,"ул. Набережная Б., д. 9, Москва",NaN,2004,22,2,Не указано
...,...,...,...,...,...,...,...,...
10877,NaN,15384.0,"б-р. Кронштадтский, д. 30, к. 3, Москва",NaN,Не заполнено,12,NaN,NaN
11407,NaN,20875.0,"ул. Новолесная, д. 18, к. 2, Москва",NaN,1966,12,NaN,NaN
12152,NaN,27341.0,"пер. Старопименовский, д. 12/6, Москва",NaN,1973,12,NaN,NaN
15213,NaN,14108.0,"ул. Климашкина, д. 9, Москва",NaN,Не заполнено,9,NaN,NaN


**year**

Ниже тоже окажутся строковые значения внутри этой колонки, но они явно не отобразятся. По-хорошему надо искать ячейки со строковым значением. Но, хорошо, что сразу нашёл, что и здесь есть ячейки с значением 'Не заполнено'.

In [ ]:
data_roh['year'].value_counts()

1960    1316
1962    1212
1961    1199
1959    1127
1963    1112
        ... 
1866       1
40         1
1861       1
22         1
1815       1
Name: year, Length: 180, dtype: int64

In [ ]:
data_roh[data_roh['year'].isnull()] #нет отсутствующих значений

,number_top,number,address,area_sqm,year,floors,entrances,apartments_flats


In [ ]:
data_roh[data_roh['year'] == 'Не заполнено']

,number_top,number,address,area_sqm,year,floors,entrances,apartments_flats
2546,NaN,38.0,"г. Зелено+2548:2788град, д. 301А, Москва",NaN,Не заполнено,Не заполнено,Не указано,Не указано
2548,NaN,686.0,"проезд. 1-й Ходынский, д. 1, Москва",1234,Не заполнено,Не заполнено,NaN,NaN
2549,NaN,724.0,"г. Зеленоград, д. 1307, Москва",NaN,Не заполнено,Не заполнено,NaN,NaN
2550,NaN,730.0,"п. Михайлово-Ярцевское, п. Секерино, д. 3, Москва",NaN,Не заполнено,Не заполнено,NaN,NaN
2551,NaN,731.0,"п. Михайлово-Ярцевское, п. Секерино, д. 4, Москва",NaN,Не заполнено,Не заполнено,NaN,NaN
...,...,...,...,...,...,...,...,...
32152,NaN,27288.0,"пер. Староконюшенный, д. 10/10, стр. 1, Москва",3066.00,Не заполнено,0,NaN,NaN
33316,NaN,NaN,"аллея. Долгопрудная, д. 1, к. 25, Москва",764.00,Не заполнено,3,Не указано,Не указано
33367,NaN,NaN,"аллея. Долгопрудная, д. 1, к. 6, Москва",213.70,Не заполнено,2,Не указано,Не указано
33369,NaN,NaN,"аллея. Долгопрудная, д. 1, к. 71, Москва",269.00,Не заполнено,2,Не указано,Не указано


In [ ]:
data_roh['year'].replace('Не заполнено', np.nan, inplace = True)

In [ ]:
data_roh[data_roh['year'].isnull()]

,number_top,number,address,area_sqm,year,floors,entrances,apartments_flats
2546,NaN,38.0,"г. Зелено+2548:2788град, д. 301А, Москва",NaN,NaN,Не заполнено,Не указано,Не указано
2548,NaN,686.0,"проезд. 1-й Ходынский, д. 1, Москва",1234,NaN,Не заполнено,NaN,NaN
2549,NaN,724.0,"г. Зеленоград, д. 1307, Москва",NaN,NaN,Не заполнено,NaN,NaN
2550,NaN,730.0,"п. Михайлово-Ярцевское, п. Секерино, д. 3, Москва",NaN,NaN,Не заполнено,NaN,NaN
2551,NaN,731.0,"п. Михайлово-Ярцевское, п. Секерино, д. 4, Москва",NaN,NaN,Не заполнено,NaN,NaN
...,...,...,...,...,...,...,...,...
32152,NaN,27288.0,"пер. Староконюшенный, д. 10/10, стр. 1, Москва",3066.00,NaN,0,NaN,NaN
33316,NaN,NaN,"аллея. Долгопрудная, д. 1, к. 25, Москва",764.00,NaN,3,Не указано,Не указано
33367,NaN,NaN,"аллея. Долгопрудная, д. 1, к. 6, Москва",213.70,NaN,2,Не указано,Не указано
33369,NaN,NaN,"аллея. Долгопрудная, д. 1, к. 71, Москва",269.00,NaN,2,Не указано,Не указано


**floors**

In [ ]:
data_roh['floors'].value_counts()

5               9419
9               5065
12              3678
14              2382
17              2212
16              2033
4               1290
8               1120
6                863
3                854
22               684
7                679
2                599
10               388
25               289
Не заполнено     251
18               206
24               203
1                201
15               186
19               139
13               138
23               125
11                99
20                77
21                56
26                20
32                19
0                 18
30                16
31                13
27                13
33                 8
28                 7
35                 7
29                 6
34                 6
40                 5
37                 4
39                 3
47                 2
49                 2
48                 2
58                 2
44                 2
38                 2
45                 1
42           

In [ ]:
data_roh[data_roh['floors'].isnull()] #нет нулевых значений

,number_top,number,address,area_sqm,year,floors,entrances,apartments_flats


In [ ]:
data_roh[data_roh['floors'] == 'Не заполнено']

,number_top,number,address,area_sqm,year,floors,entrances,apartments_flats
2546,NaN,38.0,"г. Зелено+2548:2788град, д. 301А, Москва",NaN,NaN,Не заполнено,Не указано,Не указано
2547,NaN,151.0,"г. Зеленоград, д. 1512, Москва",35015,1992.0,Не заполнено,NaN,NaN
2548,NaN,686.0,"проезд. 1-й Ходынский, д. 1, Москва",1234,NaN,Не заполнено,NaN,NaN
2549,NaN,724.0,"г. Зеленоград, д. 1307, Москва",NaN,NaN,Не заполнено,NaN,NaN
2550,NaN,730.0,"п. Михайлово-Ярцевское, п. Секерино, д. 3, Москва",NaN,NaN,Не заполнено,NaN,NaN
...,...,...,...,...,...,...,...,...
2792,NaN,31570.0,"ул. Юных Ленинцев, д. 40, к. 2, Москва",2708,NaN,Не заполнено,NaN,NaN
2793,NaN,31574.0,"ул. Юных Ленинцев, д. 44, к. 1, Москва",3613,NaN,Не заполнено,NaN,NaN
2794,NaN,31575.0,"ул. Юных Ленинцев, д. 44, к. 2, Москва",3160,NaN,Не заполнено,NaN,NaN
2795,NaN,31584.0,"ул. Юных Ленинцев, д. 48, Москва",3602,NaN,Не заполнено,NaN,NaN


In [ ]:
data_roh.loc[2546]['floors']

'Не заполнено'

In [ ]:
data_roh['floors'].replace('Не заполнено', np.nan, inplace = True)

In [ ]:
data_roh[data_roh['floors'].isnull()] 

,number_top,number,address,area_sqm,year,floors,entrances,apartments_flats
2546,NaN,38.0,"г. Зелено+2548:2788град, д. 301А, Москва",NaN,NaN,NaN,Не указано,Не указано
2547,NaN,151.0,"г. Зеленоград, д. 1512, Москва",35015,1992.0,NaN,NaN,NaN
2548,NaN,686.0,"проезд. 1-й Ходынский, д. 1, Москва",1234,NaN,NaN,NaN,NaN
2549,NaN,724.0,"г. Зеленоград, д. 1307, Москва",NaN,NaN,NaN,NaN,NaN
2550,NaN,730.0,"п. Михайлово-Ярцевское, п. Секерино, д. 3, Москва",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2792,NaN,31570.0,"ул. Юных Ленинцев, д. 40, к. 2, Москва",2708,NaN,NaN,NaN,NaN
2793,NaN,31574.0,"ул. Юных Ленинцев, д. 44, к. 1, Москва",3613,NaN,NaN,NaN,NaN
2794,NaN,31575.0,"ул. Юных Ленинцев, д. 44, к. 2, Москва",3160,NaN,NaN,NaN,NaN
2795,NaN,31584.0,"ул. Юных Ленинцев, д. 48, Москва",3602,NaN,NaN,NaN,NaN


**entrances**

In [ ]:
data_roh['entrances'].value_counts()

1             1004
4              725
3              704
2              614
5              286
6              244
Не указано     136
7              112
8              110
9               34
10              31
12              19
11              13
14              13
13               6
15               5
16               4
17               4
18               3
20               2
21               1
19               1
40               1
Name: entrances, dtype: int64

Внимание! Здесь уже есть нулевые значения в датасете, но думаю, что не страшно, если мы не задумываясь подбавим ещё нулевые значения из разряда 'Не заполнено'.

In [ ]:
data_roh[data_roh['entrances'].isnull()]

,number_top,number,address,area_sqm,year,floors,entrances,apartments_flats
136,156.0,524.0,"п. Внуковское, д. Пыхтино, д. 27, стр. по ул. ...",11989.50,2013.0,25.0,NaN,NaN
138,158.0,566.0,"п. Внуковское, д. Пыхтино, д. 16, стр. по ул. ...",12000.00,2013.0,25.0,NaN,NaN
139,159.0,567.0,"п. Внуковское, д. Пыхтино, д. 29, стр. по ул.А...",11999.40,2013.0,25.0,NaN,NaN
141,162.0,597.0,"г. Зеленоград, к. 1804Б, Москва",7718.70,2001.0,25.0,NaN,NaN
142,165.0,1929.0,"ул. Вольская 2-я, д. 1, к. 1, Москва",10808.00,2010.0,25.0,NaN,NaN
...,...,...,...,...,...,...,...,...
32152,NaN,27288.0,"пер. Староконюшенный, д. 10/10, стр. 1, Москва",3066.00,NaN,0.0,NaN,NaN
32153,NaN,29550.0,"пер. Фурманный, д. 18, Москва",5810.00,1917.0,0.0,NaN,NaN
32154,NaN,29828.0,"туп. Хомутовский, д. 4, к. 2, Москва",4466.40,1959.0,0.0,NaN,NaN
32155,NaN,29883.0,"ш. Хорошевское, д. 68, к. 4, Москва",3811.00,1962.0,0.0,NaN,NaN


In [ ]:
data_roh.loc[1944]

number_top                                   NaN
number                                   16876.0
address             ул. Льва Яшина, д. 7, Москва
area_sqm                                22835.70
year                                      2015.0
floors                                      17.0
entrances                                    NaN
apartments_flats                             NaN
Name: 1944, dtype: object

In [ ]:
data_roh[data_roh['entrances'] == 'Не указано']

,number_top,number,address,area_sqm,year,floors,entrances,apartments_flats
99,104.0,19428.0,"ул. Мосфильмовская, д. 47Б, Москва",59477.00,2017.0,29.0,Не указано,Не указано
2546,NaN,38.0,"г. Зелено+2548:2788град, д. 301А, Москва",NaN,NaN,NaN,Не указано,Не указано
2556,NaN,1238.0,"ул. Мякининская 1-я, д. 14А, стр. 1, Москва",1339,NaN,NaN,Не указано,Не указано
2566,NaN,2023.0,"ул. Мякининская 2-я, д. 20, стр. 3, Москва",1719,NaN,NaN,Не указано,Не указано
2576,NaN,2561.0,"ул. Мякининская 3-я, д. 4А, Москва",80,NaN,NaN,Не указано,Не указано
...,...,...,...,...,...,...,...,...
33391,NaN,NaN,"ул. Москворецкая, д. 9, Москва",179.10,1925.0,1.0,Не указано,5
33392,NaN,NaN,"ул. Муромская, д. 45, Москва",86.00,1959.0,1.0,Не указано,2
33393,NaN,NaN,"ул. Оренбургская, д. 5, к. 4, Москва",107.10,1954.0,1.0,Не указано,2
33394,NaN,NaN,"г. Зеленоград, проезд. 6537-й, д. 21, Москва",469.00,2009.0,1.0,Не указано,Не указано


In [ ]:
data_roh.loc[99]['entrances']

'Не указано'

In [ ]:
data_roh['entrances'].replace('Не указано', np.nan, inplace = True)

Чуточку побольше стало ненулевых значений.

In [ ]:
data_roh[data_roh['entrances'].isnull()]

,number_top,number,address,area_sqm,year,floors,entrances,apartments_flats
99,104.0,19428.0,"ул. Мосфильмовская, д. 47Б, Москва",59477.00,2017.0,29.0,NaN,Не указано
136,156.0,524.0,"п. Внуковское, д. Пыхтино, д. 27, стр. по ул. ...",11989.50,2013.0,25.0,NaN,NaN
138,158.0,566.0,"п. Внуковское, д. Пыхтино, д. 16, стр. по ул. ...",12000.00,2013.0,25.0,NaN,NaN
139,159.0,567.0,"п. Внуковское, д. Пыхтино, д. 29, стр. по ул.А...",11999.40,2013.0,25.0,NaN,NaN
141,162.0,597.0,"г. Зеленоград, к. 1804Б, Москва",7718.70,2001.0,25.0,NaN,NaN
...,...,...,...,...,...,...,...,...
33391,NaN,NaN,"ул. Москворецкая, д. 9, Москва",179.10,1925.0,1.0,NaN,5
33392,NaN,NaN,"ул. Муромская, д. 45, Москва",86.00,1959.0,1.0,NaN,2
33393,NaN,NaN,"ул. Оренбургская, д. 5, к. 4, Москва",107.10,1954.0,1.0,NaN,2
33394,NaN,NaN,"г. Зеленоград, проезд. 6537-й, д. 21, Москва",469.00,2009.0,1.0,NaN,Не указано


**apartments_flats**

In [ ]:
data_roh['apartments_flats'].value_counts()

80            216
60            167
Не указано    124
84             91
72             86
             ... 
303             1
599             1
399             1
568             1
274             1
Name: apartments_flats, Length: 535, dtype: int64

In [ ]:
data_roh[data_roh['apartments_flats'].isnull()]

,number_top,number,address,area_sqm,year,floors,entrances,apartments_flats
136,156.0,524.0,"п. Внуковское, д. Пыхтино, д. 27, стр. по ул. ...",11989.50,2013.0,25.0,NaN,NaN
138,158.0,566.0,"п. Внуковское, д. Пыхтино, д. 16, стр. по ул. ...",12000.00,2013.0,25.0,NaN,NaN
139,159.0,567.0,"п. Внуковское, д. Пыхтино, д. 29, стр. по ул.А...",11999.40,2013.0,25.0,NaN,NaN
141,162.0,597.0,"г. Зеленоград, к. 1804Б, Москва",7718.70,2001.0,25.0,NaN,NaN
142,165.0,1929.0,"ул. Вольская 2-я, д. 1, к. 1, Москва",10808.00,2010.0,25.0,NaN,NaN
...,...,...,...,...,...,...,...,...
32513,NaN,NaN,"ул. Серпуховский Вал, д. 22, к. 1, Москва",2421.00,1960.0,8.0,1.0,NaN
32574,NaN,NaN,"ул. Песчаная 2-я, д. 6, к. 5, Москва",3909.70,1951.0,6.0,2.0,NaN
32576,NaN,NaN,"ул. Авиамоторная, д. 49/1, Москва",13946.00,1928.0,6.0,14.0,NaN
32593,NaN,NaN,"ул. Кусковская, д. 16, Москва",4167.70,1937.0,6.0,5.0,NaN


In [ ]:
data_roh[data_roh['apartments_flats'] == 'Не указано']

,number_top,number,address,area_sqm,year,floors,entrances,apartments_flats
92,97.0,17964.0,"б-р. Маршала Рокоссовского, д. 6, к. 1, Москва",233822.60,2011.0,30.0,2.0,Не указано
99,104.0,19428.0,"ул. Мосфильмовская, д. 47Б, Москва",59477.00,2017.0,29.0,NaN,Не указано
123,132.0,10543.0,"ул. Губкина, д. 6, Москва",26064.00,2001.0,26.0,1.0,Не указано
213,236.0,8512.0,"ул. Верхние Поля, д. 4, Москва",23915.00,2002.0,25.0,1.0,Не указано
343,387.0,25249.0,"пр-кт. Рязанский, д. 2/1, к. 2, Москва",98632.30,2018.0,25.0,1.0,Не указано
...,...,...,...,...,...,...,...,...
33385,NaN,NaN,"п. Первомайское, д. Жуковка, д. 7(75/100), Москва",154.10,1972.0,1.0,NaN,Не указано
33388,NaN,NaN,"г. Троицк, ул. Высотная, д. 7, Москва",194.30,1967.0,1.0,NaN,Не указано
33389,NaN,NaN,"п. Первомайское, п. Птичное, ул. Заречная, д. ...",67.00,1964.0,1.0,NaN,Не указано
33394,NaN,NaN,"г. Зеленоград, проезд. 6537-й, д. 21, Москва",469.00,2009.0,1.0,NaN,Не указано


In [ ]:
data_roh.loc[92]['apartments_flats']

'Не указано'

In [ ]:
data_roh['apartments_flats'].replace('Не указано', np.nan, inplace = True)

In [ ]:
data_roh[data_roh['apartments_flats'].isnull()]

,number_top,number,address,area_sqm,year,floors,entrances,apartments_flats
92,97.0,17964.0,"б-р. Маршала Рокоссовского, д. 6, к. 1, Москва",233822.60,2011.0,30.0,2.0,NaN
99,104.0,19428.0,"ул. Мосфильмовская, д. 47Б, Москва",59477.00,2017.0,29.0,NaN,NaN
123,132.0,10543.0,"ул. Губкина, д. 6, Москва",26064.00,2001.0,26.0,1.0,NaN
136,156.0,524.0,"п. Внуковское, д. Пыхтино, д. 27, стр. по ул. ...",11989.50,2013.0,25.0,NaN,NaN
138,158.0,566.0,"п. Внуковское, д. Пыхтино, д. 16, стр. по ул. ...",12000.00,2013.0,25.0,NaN,NaN
...,...,...,...,...,...,...,...,...
33385,NaN,NaN,"п. Первомайское, д. Жуковка, д. 7(75/100), Москва",154.10,1972.0,1.0,NaN,NaN
33388,NaN,NaN,"г. Троицк, ул. Высотная, д. 7, Москва",194.30,1967.0,1.0,NaN,NaN
33389,NaN,NaN,"п. Первомайское, п. Птичное, ул. Заречная, д. ...",67.00,1964.0,1.0,NaN,NaN
33394,NaN,NaN,"г. Зеленоград, проезд. 6537-й, д. 21, Москва",469.00,2009.0,1.0,NaN,NaN


***

Сделаем ещё раз описательную статистику, хорошо если у нас изменится тип object на нужный.

In [ ]:
data_roh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33398 entries, 0 to 33397
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   number_top        1923 non-null   float64
 1   number            32157 non-null  float64
 2   address           33398 non-null  object 
 3   area_sqm          33215 non-null  object 
 4   year              33078 non-null  float64
 5   floors            33147 non-null  float64
 6   entrances         3936 non-null   float64
 7   apartments_flats  3938 non-null   object 
dtypes: float64(5), object(3)
memory usage: 2.0+ MB


area_sqm и apartments_flats пока сопротивляются, address отдельная песня

In [ ]:
data_roh['area_sqm'].astype(float)

TypeError: ignored

Затесалось значение типа datetime там, где дОлжно быть площадь.

In [ ]:
pd.to_datetime(data_roh['area_sqm'], errors='coerce')

0       NaT
1       NaT
2       NaT
3       NaT
4       NaT
         ..
33393   NaT
33394   NaT
33395   NaT
33396   NaT
33397   NaT
Name: area_sqm, Length: 33398, dtype: datetime64[ns]

In [ ]:
data_roh[data_roh['area_sqm'] == pd.NaT]

,number_top,number,address,area_sqm,year,floors,entrances,apartments_flats


In [ ]:
data_roh['area_sqm'].astype(float)

TypeError: ignored

In [ ]:
data_roh['area_sqm'].astype(float, raise_on_error=False)

TypeError: ignored

In [ ]:
isinstance(data_roh['area_sqm'][52], datetime.datetime)

False

In [ ]:
data_roh['area_sqm'].value_counts(dropna=False)

NaN         183
0.00         24
3621.00      16
3635.00      16
3637.00      15
           ... 
4717.00       1
17449.40      1
17445.40      1
8867.70       1
4466.40       1
Name: area_sqm, Length: 24107, dtype: int64

In [ ]:
data_roh[data_roh['area_sqm'].isnull()]

,number_top,number,address,area_sqm,year,floors,entrances,apartments_flats
25,27.0,13229.0,"ул. Исаковского, д. 39, к. 1, Москва",NaN,2004.0,37.0,1.0,172
70,73.0,12750.0,"ул. Ивана Бабушкина, д. 10, Москва",NaN,2008.0,31.0,1.0,143
105,110.0,25115.0,"ш. Рублевское, д. 101, Москва",NaN,2019.0,28.0,1.0,879
658,747.0,28227.0,"ул. Твардовского, д. 14, к. 1, Москва",NaN,2006.0,23.0,NaN,NaN
713,904.0,6565.0,"ул. Набережная Б., д. 9, Москва",NaN,2004.0,22.0,2.0,NaN
...,...,...,...,...,...,...,...,...
10877,NaN,15384.0,"б-р. Кронштадтский, д. 30, к. 3, Москва",NaN,NaN,12.0,NaN,NaN
11407,NaN,20875.0,"ул. Новолесная, д. 18, к. 2, Москва",NaN,1966.0,12.0,NaN,NaN
12152,NaN,27341.0,"пер. Старопименовский, д. 12/6, Москва",NaN,1973.0,12.0,NaN,NaN
15213,NaN,14108.0,"ул. Климашкина, д. 9, Москва",NaN,NaN,9.0,NaN,NaN


Ну вот выяснили. Заменим NaT на NaN.

In [ ]:
data_roh['area_sqm'].replace(pd.NaT, np.nan, inplace=True)

In [ ]:
#pd.to_datetime(data_roh['area_sqm'], errors='coerce')

In [ ]:
data_roh['area_sqm'].sample(10)

9741      5055.80
9726     14524.00
28636     3024.00
1611     25705.00
7948     12380.00
25121     2922.30
24660     2905.00
21926     3558.60
17014     8011.80
32860     2570.50
Name: area_sqm, dtype: object

In [ ]:
type(data_roh['area_sqm'][50])

str

In [ ]:
#data_roh['area_sqm'] = data_roh_['Площадь м2']

**ООО Вот находка-то есть здесь и str значения и int и float и datetime.datetime**

In [ ]:
data_roh['area_sqm'].apply(type).value_counts()

<class 'str'>                  33308
<class 'int'>                     48
<class 'float'>                   39
<class 'datetime.datetime'>        3
Name: area_sqm, dtype: int64

In [ ]:
type(data_roh['area_sqm']) == 'datetime.datetime'

False

In [ ]:
data_roh['area_sqm'].apply(type).value_counts().index[3]

datetime.datetime

In [ ]:
data_roh[data_roh['area_sqm'].apply(type) == int]

,number_top,number,address,area_sqm,year,floors,entrances,apartments_flats
2547,NaN,151.0,"г. Зеленоград, д. 1512, Москва",35015,1992.0,NaN,NaN,NaN
2548,NaN,686.0,"проезд. 1-й Ходынский, д. 1, Москва",1234,NaN,NaN,NaN,NaN
2556,NaN,1238.0,"ул. Мякининская 1-я, д. 14А, стр. 1, Москва",1339,NaN,NaN,NaN,NaN
2557,NaN,1239.0,"ул. Мякининская 1-я, д. 14А, стр. 2, Москва",1339,NaN,NaN,NaN,NaN
2559,NaN,2016.0,"ул. Мякининская 2-я, д. 3А, стр. 1, Москва",1199,NaN,NaN,NaN,NaN
2566,NaN,2023.0,"ул. Мякининская 2-я, д. 20, стр. 3, Москва",1719,NaN,NaN,NaN,NaN
2568,NaN,2025.0,"ул. Мякининская 2-я, д. 50, стр. 2, Москва",330,NaN,NaN,NaN,NaN
2569,NaN,2026.0,"ул. Мякининская 2-я, д. 50А, стр. 1, Москва",330,NaN,NaN,NaN,NaN
2576,NaN,2561.0,"ул. Мякининская 3-я, д. 4А, Москва",80,NaN,NaN,NaN,NaN
2578,NaN,2563.0,"ул. Мякининская 3-я, д. 10, стр. 2, литера А, ...",1018,NaN,NaN,NaN,NaN


In [ ]:
data_roh[data_roh['area_sqm'].apply(type) == float]

,number_top,number,address,area_sqm,year,floors,entrances,apartments_flats
2558,NaN,1240.0,"ул. Мякининская 1-я, д. 14А, Москва",32.7,NaN,NaN,NaN,NaN
2560,NaN,2017.0,"ул. Мякининская 2-я, д. 3А, Москва",44.5,NaN,NaN,NaN,NaN
2561,NaN,2018.0,"ул. Мякининская 2-я, д. 19А, литера А, Москва",1708.6,NaN,NaN,NaN,NaN
2562,NaN,2019.0,"ул. Мякининская 2-я, д. 19А, Москва",33.2,NaN,NaN,NaN,NaN
2563,NaN,2020.0,"ул. Мякининская 2-я, д. 20А, (3/4 доли), Москва",147.45,NaN,NaN,NaN,NaN
2564,NaN,2021.0,"ул. Мякининская 2-я, д. 20, стр. 1, Москва",165.9,NaN,NaN,NaN,NaN
2565,NaN,2022.0,"ул. Мякининская 2-я, д. 20, стр. 2, Москва",67.3,NaN,NaN,NaN,NaN
2567,NaN,2024.0,"ул. Мякининская 2-я, д. 38, стр. 2, Москва",2916.5,NaN,NaN,NaN,NaN
2570,NaN,2027.0,"ул. Мякининская 2-я, д. 50, стр. 1, Москва",62.3,NaN,NaN,NaN,NaN
2571,NaN,2028.0,"ул. Мякининская 2-я, д. 50А, Москва",22.3,NaN,NaN,NaN,NaN


**А, добился!**

In [ ]:
data_roh[data_roh['area_sqm'].apply(type) == datetime.datetime]

,number_top,number,address,area_sqm,year,floors,entrances,apartments_flats
15809,NaN,18344.0,"ул. Машкова, д. 1, Москва",1964-04-01 00:00:00,2002.0,9.0,NaN,NaN
30505,NaN,29538.0,"пер. Фурманный, д. 2/7, стр. 1, Москва",1940-02-01 00:00:00,2000.0,4.0,NaN,NaN
33287,NaN,NaN,"пер. Фурманный, д. 2/7, стр. 1, Москва",1940-02-01 00:00:00,2000.0,4.0,1.0,10


In [ ]:
data_roh['area_sqm'][15809]

datetime.datetime(1964, 4, 1, 0, 0)

Заменил с указанием конкретной ячейки, подействовало на все три

In [ ]:
data_roh['area_sqm'].replace('datetime.datetime(1964, 4, 1, 0, 0)', np.nan, inplace = True)

In [ ]:
data_roh['area_sqm'][15809]

nan

In [ ]:
data_roh['area_sqm'][33287]

nan

In [ ]:
data_roh['area_sqm'][3327] #Прежние значения

'5130.00'

Бессмысленно datetime.datetime переводить в float... Данных тоже немного, давайте просто без цикла и ухищрений заменим конкретную ячейку на np.nan

In [ ]:
data_roh['area_sqm'].replace('Не заполнено', np.nan, inplace = True)

In [ ]:
data_roh['area_sqm'].apply(float)

0         54804.10
1         53253.20
2        256740.60
3        137500.80
4        341272.69
           ...    
33393       107.10
33394       469.00
33395       133.00
33396      8462.30
33397      3811.00
Name: area_sqm, Length: 33398, dtype: float64

In [ ]:
data_roh['area_sqm'].apply(type).value_counts()

<class 'str'>      33125
<class 'float'>      225
<class 'int'>         48
Name: area_sqm, dtype: int64

In [ ]:
''.isnumeric()

False

In [ ]:
tqr = data_roh[data_roh['area_sqm'].apply(type) == str]['area_sqm'].tolist()

In [ ]:
for tq in tqr:
    if (tq.isalpha())
    print('f')

SyntaxError: ignored

In [ ]:
data_roh['area_sqm'].astype(float)

0         54804.10
1         53253.20
2        256740.60
3        137500.80
4        341272.69
           ...    
33393       107.10
33394       469.00
33395       133.00
33396      8462.30
33397      3811.00
Name: area_sqm, Length: 33398, dtype: float64

In [ ]:
type(data_roh_['Площадь м2'][50])

str

In [ ]:
lol = data_roh['area_sqm'][50]
float(lol)

17356.0

In [ ]:
float(np.nan)

nan

In [ ]:
st

In [ ]:
data_roh['area_sqm'].describe()

count     33215
unique    24106
top        0.00
freq         24
Name: area_sqm, dtype: object

In [ ]:
data_roh['apartments_flats'].astype(float) #хорошо, что это вызвало ошибку.
#По идее у нас находятся int значения в этой колонке. 

ValueError: ignored

Кто-то взял и написал с маленькой буквы 'не указано', хотя мы уже имели дело с 'Не указано'. Хорошо, заменим.

In [ ]:
data_roh[data_roh['apartments_flats'] == 'не указано']

,number_top,number,address,area_sqm,year,floors,entrances,apartments_flats
4621,NaN,634.0,"кв-л. Волжский Бульвар 114 А, к. 6, Москва",5595.00,1989.0,16.0,1.0,не указано
8021,NaN,20024.0,"пр-кт. Нахимовский, д. 11, к. 1, Москва",6553.00,1973.0,14.0,1.0,не указано
9525,NaN,4119.0,"ул. Академика Глушко, д. 14, к. 2, стр. 9, 10,...",4173.80,1993.0,12.0,3.0,не указано
11825,NaN,24716.0,"ул. Ратная, д. 8, к. 1, Москва",4343.00,1992.0,12.0,4.0,не указано
15245,NaN,14353.0,"пр-кт. Комсомольский, д. 15, стр. 2, Москва",6712.00,1965.0,9.0,4.0,не указано
18485,NaN,14832.0,"ул. Косыгина, д. 2, к. 1, Москва",22566.00,2010.0,8.0,2.0,не указано
27545,NaN,24415.0,"ул. Пулковская, д. 7, Москва",3004.00,1970.0,5.0,3.0,не указано
28175,NaN,27022.0,"ул. Спортивная, д. 6/18, Москва",3199.00,1960.0,5.0,4.0,не указано
29165,NaN,31239.0,"ш. Щелковское, д. 73, Москва",3220.00,1963.0,5.0,1.0,не указано
29915,NaN,16343.0,"ул. Леси Украинки, д. 6, к. 2, Москва",1300.40,0.0,4.0,2.0,не указано


In [ ]:
data_roh.loc[4621]['apartments_flats']

'не указано'

In [ ]:
data_roh['apartments_flats'].replace('не указано', np.nan, inplace = True)

In [ ]:
data_roh['apartments_flats'].describe()

count     3938
unique     534
top         80
freq       216
Name: apartments_flats, dtype: int64

In [ ]:
data_roh['apartments_flats'].apply(type).value_counts()

<class 'float'>    33398
Name: apartments_flats, dtype: int64

Всё, числовое значение!

Ещё раз пройдёмся.

In [ ]:
data_roh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33398 entries, 0 to 33397
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   number_top        1923 non-null   float64
 1   number            32157 non-null  float64
 2   address           33398 non-null  object 
 3   area_sqm          33212 non-null  object 
 4   year              33078 non-null  float64
 5   floors            33147 non-null  float64
 6   entrances         3936 non-null   float64
 7   apartments_flats  3913 non-null   float64
dtypes: float64(6), object(2)
memory usage: 2.0+ MB


****

In [ ]:
profile_roh = ProfileReport(data_roh, title = 'Register of houses', html = {'style':{'full_width':True}})

In [ ]:
 profile_roh.to_notebook_iframe()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
profile_roh.to_file("/content/drive/My Drive/Colab Notebooks/lkl2022/10/data/profile_report_data_roh.html")


Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Сохраним датасет в файл.

In [ ]:
data_roh.to_csv('/content/drive/My Drive/Colab Notebooks/lkl2022/Q10/data/renew_roh.csv')

Убедимся, что всё работает. Загрузим сохранённый файл.

In [ ]:
ddf = pd.read_csv('/content/drive/My Drive/Colab Notebooks/lkl2022/Q10/data/renew_roh.csv')

In [ ]:
ddf.sample(5)

,Unnamed: 0,number_top,number,address,area_sqm,year,floors,entrances,apartments_flats
24888,24888,NaN,15107.0,"ул. Краснодонская, д. 4, Москва",3160.4,1967.0,5.0,NaN,NaN
10138,10138,NaN,10050.0,"ул. Героев Панфиловцев, д. 41, к. 1, Москва",5585.6,1968.0,12.0,NaN,NaN
25812,25812,NaN,18438.0,"ул. Мельникова, д. 24, Москва",4563.6,1960.0,5.0,NaN,NaN
20553,20553,NaN,18.0,"ул. Типографская, д. 24, Москва",4427.0,1983.0,5.0,NaN,NaN
22858,22858,NaN,7902.0,"ул. Вавилова, д. 20, Москва",2549.0,1960.0,5.0,NaN,NaN


In [ ]:
ddf

,Unnamed: 0,number_top,number,address,area_sqm,year,floors,entrances,apartments_flats
0,0,1.0,18803.0,"пр-кт. Мира, д. 188 Б, к. 1, Москва",54804.10,2014.0,58.0,1.0,370.0
1,1,2.0,18805.0,"пр-кт. Мира, д. 188Б, к. 3, Москва",53253.20,2015.0,58.0,1.0,356.0
2,2,3.0,29988.0,"пер. Чапаевский, д. 3, Москва",256740.60,2006.0,57.0,14.0,992.0
3,3,4.0,19390.0,"ул. Мосфильмовская, д. 8, Москва",137500.80,2011.0,53.0,2.0,564.0
4,4,5.0,19431.0,"ул. Мосфильмовская, д. 70, Москва",341272.69,2006.0,50.0,7.0,1046.0
...,...,...,...,...,...,...,...,...,...
33393,33393,NaN,NaN,"ул. Оренбургская, д. 5, к. 4, Москва",107.10,1954.0,1.0,NaN,2.0
33394,33394,NaN,NaN,"г. Зеленоград, проезд. 6537-й, д. 21, Москва",469.00,2009.0,1.0,NaN,NaN
33395,33395,NaN,NaN,"ул. Спортивная, д. 10, Москва",133.00,1940.0,1.0,NaN,NaN
33396,33396,NaN,NaN,"проезд. Коломенский, д. 8, к. 1, Москва",8462.30,1982.0,0.0,2.0,128.0


In [ ]:
ddf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33398 entries, 0 to 33397
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        33398 non-null  int64  
 1   number_top        1923 non-null   float64
 2   number            32157 non-null  float64
 3   address           33398 non-null  object 
 4   area_sqm          33212 non-null  float64
 5   year              33078 non-null  float64
 6   floors            33147 non-null  float64
 7   entrances         3936 non-null   float64
 8   apartments_flats  3913 non-null   float64
dtypes: float64(7), int64(1), object(1)
memory usage: 2.3+ MB


Прикольно, при сохранении в csv получилось преобразовать area_sqm  в float.

In [ ]:
del ddf['Unnamed: 0']

In [ ]:
ddf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33398 entries, 0 to 33397
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   number_top        1923 non-null   float64
 1   number            32157 non-null  float64
 2   address           33398 non-null  object 
 3   area_sqm          33212 non-null  float64
 4   year              33078 non-null  float64
 5   floors            33147 non-null  float64
 6   entrances         3936 non-null   float64
 7   apartments_flats  3913 non-null   float64
dtypes: float64(7), object(1)
memory usage: 2.0+ MB


In [ ]:
profile_rohnew = ProfileReport(ddf, title = 'Register of houses', html = {'style':{'full_width':True}})

In [ ]:
 profile_rohnew.to_notebook_iframe()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
profile_rohnew.to_file("/content/drive/My Drive/Colab Notebooks/lkl2022/10/data/profile_report_data_rohnew.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]